In [1]:
import argparse
import csv
import os
import sys
import numpy as np
import matplotlib
import matplotlib.font_manager as fm
import matplotlib.patheffects as pe
import pandas as pd
import sys
import textwrap

from itertools import product, combinations
from matplotlib import cm
from matplotlib.ticker import FormatStrFormatter
from matplotlib import pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from mpl_toolkits.mplot3d import axes3d
import matplotlib.transforms as mtransforms
from matplotlib.patches import Circle, PathPatch
import mpl_toolkits.mplot3d.art3d as art3d
from matplotlib.text import TextPath
from matplotlib.transforms import Affine2D

%matplotlib widget

In [ ]:
matplotlib.use('pgf')
matplotlib.rcParams['pgf.texsystem'] = 'pdflatex'
matplotlib.rcParams['text.usetex'] = True
matplotlib.rcParams['text.latex.preamble'] = [ r'\usepackage{siunitx}',
                                               r'\sisetup{detect-all}',
                                               r'\usepackage{helvet}',
                                               r'\usepackage{sansmath}',
                                               r'\sansmath']

In [4]:
def create_csv_dict(csv_table):
    '''
    Reads a table containing Franck-Condon trajectory
    information. Creates a dictionary where the keys
    are the titles of the columns and the values are
    list taken from the corresponding columns entries.
    '''

    with open(csv_table, 'r') as ct:
        dataframe = pd.read_csv(ct, header=0)

    return dataframe

def slice_pandas_series(pd_series, lim):
    '''
    '''
    if float(pd_series[-1:]) > float(lim):
        for index, value in enumerate(pd_series):
            if value > lim:
                slicing_index = index
                break

    return slicing_index

In [5]:
csv = 'ar7_f13_results_FC.csv'
df = create_csv_dict(csv)

In [6]:
df.keys()

Index(['Time Scaled', 'S0 CASSCF 2R', 'S1 CASSCF 2R', 'S0 CASSCF 2R Scaled',
       'S1 CASSCF 2R Scaled', 'S0 CASSCF 2R Scaled CASPT2',
       'S1 CASSCF 2R Scaled CASPT2', 'S0 CASSCF Energy', 'S1 CASSCF Energy',
       'S2 CASSCF Energy', 'S0 CASPT2 Energy', 'S1 CASPT2 Energy',
       'S2 CASPT2 Energy', 'Scaled S0 CASSCF Energy',
       'Scaled S1 CASSCF Energy', 'Scaled S2 CASSCF Energy',
       'Scaled S0 CASPT2 Energy', 'Scaled S1 CASPT2 Energy', 'Unnamed: 18',
       'Scaled S2 CASPT2 Energy', 'S0 Mulliken Charge', 'S1 Mulliken Charge',
       'S2 Mulliken Charge', 'Oscillator Strength S0S1',
       'Oscillator Strength S0S2', 'Oscillator Strength S1S2', 'Reactive TORS',
       'Reactive HOOP', 'HOOP in phase', 'TAU overlap', 'full-BLA', 'beta-BLA',
       'rpsb-BLA'],
      dtype='object')

In [7]:
len(df.keys())

33

In [8]:
df.head()

,Time Scaled,S0 CASSCF 2R,S1 CASSCF 2R,S0 CASSCF 2R Scaled,S1 CASSCF 2R Scaled,S0 CASSCF 2R Scaled CASPT2,S1 CASSCF 2R Scaled CASPT2,S0 CASSCF Energy,S1 CASSCF Energy,S2 CASSCF Energy,...,Oscillator Strength S0S1,Oscillator Strength S0S2,Oscillator Strength S1S2,Reactive TORS,Reactive HOOP,HOOP in phase,TAU overlap,full-BLA,beta-BLA,rpsb-BLA
0,0.0000,-869.021421,-868.908881,0.0,70.6,0.0,47.2,-869.021843,-868.907728,-868.880461,...,1.429031,0.329143,0.051163,-15.070,-9.261,-2.9045,-12.1655,0.090,0.090,0.114
1,1.2235,-869.021621,-868.910283,-0.1,69.7,-0.1,46.6,-869.021488,-868.910346,-868.884860,...,1.485390,0.303284,0.048310,-15.069,-9.345,-2.8620,-12.2070,0.081,0.082,0.101
2,2.4470,-869.021185,-868.913040,0.1,68.0,0.1,45.4,-869.020138,-868.913349,-868.890678,...,1.584949,0.251891,0.040584,-15.067,-9.484,-2.7915,-12.2755,0.067,0.069,0.081
3,3.6705,-869.019683,-868.916329,1.1,65.9,0.7,44.1,-869.017405,-868.915721,-868.896529,...,1.733408,0.164335,0.025491,-15.069,-9.680,-2.6945,-12.3745,0.050,0.053,0.057
4,4.8940,-869.016791,-868.919123,2.9,64.2,1.9,42.9,-869.013177,-868.916998,-868.901025,...,1.912635,0.050718,0.006151,-15.077,-9.931,-2.5730,-12.5040,0.030,0.034,0.031


In [9]:
df.iloc[:,20]

0      0.5706
1      0.5607
2      0.5449
3      0.5243
4      0.5002
        ...  
372       NaN
373       NaN
374       NaN
375       NaN
376       NaN
Name: S0 Mulliken Charge, Length: 377, dtype: float64

In [10]:
plt.scatter(df.iloc[:,32], df.iloc[:,20])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [31]:
def plot_bla_tor_hop_3d(traj_csv_dict_1,
                        traj_csv_dict_2,
                        traj_csv_dict_3,
                        traj_csv_dict_4,
                        project_name, rpath, xlim=None, ylim=None):
    '''
    @author: Leonardo Barneschi
    Create a plot of the Retinal Mulliken Charges.
    '''

    fig = plt.figure()
    ax = plt.subplot(projection='3d')#**gen_params['canvas_dimension'])

    #ax.set_aspect("equal")

    ax.w_xaxis.set_pane_color((0.96, 0.96, 0.94, 1.0))
    ax.w_yaxis.set_pane_color((0.96, 0.96, 0.94, 1.0))
    ax.w_zaxis.set_pane_color((0.96, 0.96, 0.94, 1.0))

    ax.xaxis.pane.set_edgecolor('#000000')
    ax.yaxis.pane.set_edgecolor('#000000')
    ax.zaxis.pane.set_edgecolor('#000000')
    #ax.zaxis.pane.set_edgecolor('#D0D0D0')

    ax.xaxis.pane.set_alpha(1)
    ax.yaxis.pane.set_alpha(1)
    ax.zaxis.pane.set_alpha(1)

    # >>>> Set X and Y axis titles.
    #ax.set_xlabel('Torsion')#, **gen_params['x_title_dimension'], labelpad=10)
    #ax.set_ylabel('HOOP')#, **gen_params['y_title_dimension'], labelpad=10)
    #ax.set_zlabel('BLA')#, **gen_params['z_title_dimension'], labelpad=10)
   
    ax.set_xlim(-170, -90)
    ax.set_ylim(-0.15,0.15)
    ax.set_zlim(0,140)
    ax.set_xlabel('${\mathrm{C_{12}-C_{13}=C_{14}-C_{15}\hspace{0.05in}(degrees)}}$')
    ax.set_ylabel(('${\mathrm{BLA\hspace{0.03in}(angstrom)}}$')
    ax.set_zlabel('${\mathrm{Relative\hspace{0.03in}Energy\hspace{0.03in}(kcal\hspace{0.03in}mol^{-1})}}$')

    ax.scatter(traj_csv_dict_1['S0 Mulliken Charge'], traj_csv_dict_1['S1 Mulliken Charge'], traj_csv_dict_1['rpsb-BLA'], label='qar1')
    ax.scatter(traj_csv_dict_2['S0 Mulliken Charge'], traj_csv_dict_2['S1 Mulliken Charge'], traj_csv_dict_2['rpsb-BLA'], label='qar2')
    ax.scatter(traj_csv_dict_3['S0 Mulliken Charge'], traj_csv_dict_3['S1 Mulliken Charge'], traj_csv_dict_3['rpsb-BLA'], label='ar5')
    ax.scatter(traj_csv_dict_4['S0 Mulliken Charge'], traj_csv_dict_4['S1 Mulliken Charge'], traj_csv_dict_4['rpsb-BLA'], label='ar7')

    ax.legend()

In [32]:
traj_csv_dict_1 = "qar1_f13_results_FC.csv"
traj_csv_dict_2 = "qar2_f13_results_FC.csv"
traj_csv_dict_3 = "ar5_f13_results_FC.csv"
traj_csv_dict_4 = "ar7_f13_results_FC.csv"

project_name  = "debug_mode"
rpath = "."

csv_dict_1 = create_csv_dict(traj_csv_dict_1)
csv_dict_2 = create_csv_dict(traj_csv_dict_2)
csv_dict_3 = create_csv_dict(traj_csv_dict_3)
csv_dict_4 = create_csv_dict(traj_csv_dict_4)

plot_bla_tor_hop_3d(csv_dict_1, csv_dict_2, csv_dict_3, csv_dict_4, project_name, rpath)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [135]:
csv_dict_1['Reactive TORS']

0     -15.520
1     -15.521
2     -15.525
3     -15.533
4     -15.550
        ...  
368   -20.886
369   -20.774
370       NaN
371       NaN
372       NaN
Name: Reactive TORS, Length: 373, dtype: float64